In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [20]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_out = pd.read_csv("test.csv")

In [3]:
def data_clean(ds):
    ds['Embarked'].replace({'S':1, 'C':2, 'Q':3}, inplace = True)
    ds['Embarked-S'] = 0
    ds['Embarked-C'] = 0
    ds['Embarked-Q'] = 0
    
    for index, row in ds.iterrows():
        if row['Embarked'] == 1:
            ds.loc[index,'Embarked-S'] = 1
        elif row['Embarked'] == 2:
            ds.loc[index,'Embarked-C'] = 1
        elif row['Embarked'] == 3:
            ds.loc[index,'Embarked-Q'] = 1
    
    ds['Pclass-1'] = 0
    ds['Pclass-2'] = 0
    ds['Pclass-3'] = 0
    
    for index, row in ds.iterrows():
        if row['Pclass'] == 1:
            ds.loc[index,'Pclass-1'] = 1
        elif row['Pclass'] == 2:
            ds.loc[index,'Pclass-2'] = 1
        elif row['Pclass'] == 3:
            ds.loc[index,'Pclass-3'] = 1

    #Title        
    ds['Title'] = test.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    ds['Col'] = 0
    ds['Dona'] = 0
    ds['Dr'] = 0
    ds['Master'] = 0
    ds['Miss'] = 0
    ds['Mr'] = 0
    ds['Mrs'] = 0
    ds['Ms'] = 0
    ds['Rev'] = 0
    
    for index, row in ds.iterrows():
        if row['Title'] == 'Col':
            ds.loc[index,'Col'] = 1
        elif row['Title'] == 'Dona':
            ds.loc[index,'Dona'] = 1
        elif row['Title'] == 'Dr':
            ds.loc[index,'Dr'] = 1
        elif row['Title'] == 'Master':
            ds.loc[index,'Master'] = 1
        elif row['Title'] == 'Miss':
            ds.loc[index,'Miss'] = 1
        elif row['Title'] == 'Mr':
            ds.loc[index,'Mr'] = 1
        elif row['Title'] == 'Mrs':
            ds.loc[index,'Mrs'] = 1
        elif row['Title'] == 'Ms':
            ds.loc[index,'Ms'] = 1
        elif row['Title'] == 'Rev':
            ds.loc[index,'Rev'] = 1
        else:
            None

    #end of Title
    ds['Sex'].replace({'male':0,'female':1}, inplace = True)
    ds.drop(['PassengerId','Ticket', 'Fare', 'Cabin', 'Name', 'Embarked', 'Pclass', 'Title'], axis=1,inplace=True)
    ds["Age"] = ds.groupby(['Sex','Pclass-1', 'Pclass-2', 'Pclass-3','SibSp','Parch'])['Age'].transform(lambda x: x.fillna(x.mean()))
    ds["Age"] = ds.groupby(['Sex','Pclass-1', 'Pclass-2', 'Pclass-3'])['Age'].transform(lambda x: x.fillna(x.mean()))
    
    for index, row in ds.iterrows():
        ds.loc[index,'Age'] = round(ds.loc[index,'Age'])

    #ds["Embarked"] = ds.groupby(['Sex','Pclass'])['Embarked'].transform(lambda x: x.fillna(x.median()))
    return ds

In [21]:
train = data_clean(train)
test = data_clean(test)

In [5]:
train.head(50)

,Survived,Sex,Age,SibSp,Parch,Embarked-S,Embarked-C,Embarked-Q,Pclass-1,Pclass-2,Pclass-3,Col,Dona,Dr,Master,Miss,Mr,Mrs,Ms,Rev
0,0,0,22.0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0
1,1,1,38.0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0
2,1,1,26.0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0
3,1,1,35.0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0
4,0,0,35.0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0
5,0,0,29.0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0
6,0,0,54.0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
7,0,0,2.0,3,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0
8,1,1,27.0,0,2,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0
9,1,1,14.0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0


In [6]:
X = train.drop(['Survived'], axis='columns')
y = train['Survived']

In [7]:
test[test.isnull().any(axis=1)]

,Sex,Age,SibSp,Parch,Embarked-S,Embarked-C,Embarked-Q,Pclass-1,Pclass-2,Pclass-3,Col,Dona,Dr,Master,Miss,Mr,Mrs,Ms,Rev


In [8]:
##Separando dataset para treino e teste
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [9]:
##Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model1 = LogisticRegression(solver='lbfgs', max_iter=100000)
model1.fit(X_train, y_train)
pred = model1.predict(X_test)
log_acc = accuracy_score(pred, y_test)
print(log_acc)

0.8212290502793296


In [10]:
##RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
model2=RandomForestClassifier(n_estimators =500)
model2.fit(X_train,y_train)
pred_2=model2.predict(X_test)
rcf=accuracy_score(pred_2,y_test);
print(rcf)    

0.8379888268156425


In [11]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
model4=KNeighborsClassifier(n_neighbors=9)
model4.fit(X_train,y_train)
pred_4=model4.predict(X_test)
knn=accuracy_score(pred_4,y_test);
print(knn)

0.776536312849162


C:\Users\Danilo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [12]:
test.head(5)

,Sex,Age,SibSp,Parch,Embarked-S,Embarked-C,Embarked-Q,Pclass-1,Pclass-2,Pclass-3,Col,Dona,Dr,Master,Miss,Mr,Mrs,Ms,Rev
0,0,34.0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0
1,1,47.0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0
2,0,62.0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0
3,0,27.0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0
4,1,22.0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0


In [13]:
X.head()

,Sex,Age,SibSp,Parch,Embarked-S,Embarked-C,Embarked-Q,Pclass-1,Pclass-2,Pclass-3,Col,Dona,Dr,Master,Miss,Mr,Mrs,Ms,Rev
0,0,22.0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0
1,1,38.0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0
2,1,26.0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0
3,1,35.0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0
4,0,35.0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0


In [14]:
model1.fit(X,y)
pred = model1.predict(test) #Logistic Regression

In [15]:
sub = pd.DataFrame({"PassengerId": test_out["PassengerId"],"Survived":pred})

sub.to_csv('sub.csv',index=False)

In [16]:
test.groupby(['Sex']).mean()

,Age,SibSp,Parch,Embarked-S,Embarked-C,Embarked-Q,Pclass-1,Pclass-2,Pclass-3,Col,Dona,Dr,Master,Miss,Mr,Mrs,Ms,Rev
Sex,,,,,,,,,,,,,,,,,,
0,29.699248,0.379699,0.274436,0.684211,0.233083,0.082707,0.214286,0.236842,0.548872,0.007519,0.000000,0.003759,0.078947,0.000000,0.902256,0.000000,0.000000,0.007519
1,29.631579,0.565789,0.598684,0.578947,0.263158,0.157895,0.328947,0.197368,0.473684,0.000000,0.006579,0.000000,0.000000,0.513158,0.000000,0.473684,0.006579,0.000000


In [22]:
pd.crosstab(test['Title'], test['Survived'])
#create new col with summarize cols 'SibSp','Parch' 

KeyError: 'Title'